# PHASE_5_CAPSTONE_PROJECT

##  TRANSFER LEARNING - RESNET50

### Data source

The data source consists of over 44 thousand fashion images from Kaggle.

Source link: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset

### Target

NICEON, the multi-brand fashion company, is excited to introduce a cutting-edge feature that allows customers to perform image-based searches. With this new capability, the website can identify product types, empowering users to explore the top 5 closely matched products, resulting in a captivating and personalized shopping experience.

### Resnet50 introduction

In [1]:
import pandas as pd
import numpy as np
import csv
import json
import glob
import os
import shutil
import random
import pickle
import h5py
import hdf5storage
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from IPython.display import display
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras import layers
from keras.models import Sequential
from keras.layers import Reshape

from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array
from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import Dropout
from keras.models import load_model


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

# a new layer that rescales/normalizes the activations after each layer.
# has a weak regularization effect. also allows 

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers, regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Resizing, RandomFlip, RandomRotation
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess


%matplotlib inline


In [2]:
# Directory path
train_data_dir = '/Users/thaophan/Documents/Flatiron/Phase5_Project/Capstone_fashion_label_classification_from_images/Data/Train_Test_Validation/Train'
val_data_dir = '/Users/thaophan/Documents/Flatiron/Phase5_Project/Capstone_fashion_label_classification_from_images/Data/Train_Test_Validation/Validation'
test_data_dir = '/Users/thaophan/Documents/Flatiron/Phase5_Project/Capstone_fashion_label_classification_from_images/Data/Train_Test_Validation/Test'




# Define the image size and number of classes
image_size_224 = (224, 224)
num_classes = 10 

# specialized resnet50 preprocesser
resnet50_processor = tf.keras.applications.resnet50.preprocess_input

# Create the train_datagen with vgg_processor
train_datagen_resnet50 = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.3, 
                                   zoom_range=0.1, 
                                   horizontal_flip=False,
                                   preprocessing_function=resnet50_processor)

# Create the val_datagen with vgg_processor
val_datagen_resnet50 = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function=resnet50_processor)

# Create the test_datagen with vgg_processor
test_datagen_resnet50 = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=resnet50_processor)

# ...

# Flow data from the directory using the data generators
train_generator_resnet50 = train_datagen_resnet50.flow_from_directory(directory=train_data_dir,
                                                    target_size=image_size_224,
                                                    class_mode='categorical',
                                                    batch_size=64,
                                                    shuffle=True,
                                                    classes=[str(i) for i in range(num_classes)])

val_generator_resnet50 = val_datagen_resnet50.flow_from_directory(directory=val_data_dir,
                                                target_size=image_size_224,
                                                class_mode='categorical',
                                                batch_size=64,
                                                shuffle=True,
                                                classes=[str(i) for i in range(num_classes)])

test_generator_resnet50 = test_datagen_resnet50.flow_from_directory(directory=test_data_dir,
                                                  target_size=image_size_224,
                                                  class_mode='categorical',
                                                  batch_size=64,
                                                  shuffle=False,
                                                  classes=[str(i) for i in range(num_classes)])


Found 14782 images belonging to 10 classes.
Found 4935 images belonging to 10 classes.
Found 4926 images belonging to 10 classes.


### ResNet-50 model

In [3]:
base_resnet50_model = tf.keras.applications.resnet50.ResNet50(include_top=False, 
                                                            weights = 'imagenet')
for layers in base_resnet50_model.layers:
  layers.trainable = False

In [4]:
base_resnet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, None, None, 64)       16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, None, None, 64)       256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, None, None, 64)       0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, None, None, 512)      66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, None, None, 512)      2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, None, None, 512)      0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, None, None, 256)      590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, None, None, 256)      1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, None, None, 256)      0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, None, None, 1024)     525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, None, None, 1024)     0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, None, None, 256)      262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, None, None, 256)      1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, None, None, 1024)     4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, None, None, 1024)     0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, None, None, 1024)     0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, None, None, 512)      0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, None, None, 512)      2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, None, None, 512)      2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, None, None, 512)      0         ['conv5_block3_2_bn[0][0]']   
 ation)   

In [5]:
len(base_resnet50_model.layers)

175

In [12]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [13]:
resnet50_model = custom_classifier(base_resnet50_model, num_classes)

In [14]:
resnet50_model.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [15]:
history_resnet50_model = resnet50_model.fit(train_generator_resnet50,
                                                  steps_per_epoch=len(train_generator_resnet50),
                                                  epochs=50,
                                                  validation_data=val_generator_resnet50,
                                                  validation_steps=len(val_generator_resnet50))

Epoch 1/50
231/231 [==============================] - 514s 2s/step - loss: 1.7504 - accuracy: 0.3581 - auc: 0.8193 - precision: 0.6010 - recall: 0.0662 - val_loss: 1.3616 - val_accuracy: 0.4823 - val_auc: 0.9067 - val_precision: 0.6855 - val_recall: 0.2067
Epoch 2/50
231/231 [==============================] - 508s 2s/step - loss: 1.4608 - accuracy: 0.4428 - auc: 0.8816 - precision: 0.6690 - recall: 0.1475 - val_loss: 1.1957 - val_accuracy: 0.5658 - val_auc: 0.9244 - val_precision: 0.8271 - val_recall: 0.2501
Epoch 3/50
231/231 [==============================] - 1673s 7s/step - loss: 1.3469 - accuracy: 0.4855 - auc: 0.9001 - precision: 0.6998 - recall: 0.2075 - val_loss: 1.1257 - val_accuracy: 0.6087 - val_auc: 0.9328 - val_precision: 0.8028 - val_recall: 0.3027
Epoch 4/50
231/231 [==============================] - 805s 3s/step - loss: 1.2572 - accuracy: 0.5304 - auc: 0.9131 - precision: 0.7269 - recall: 0.2614 - val_loss: 1.0468 - val_accuracy: 0.6211 - val_auc: 0.9414 - val_precision:

#### Save model and history

In [16]:
resnet50_model.save('resnet50_model_50epoch.h5')

/Users/seanluong/miniforge3/envs/Thaoworkingenvironment/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
hdf5storage.savemat("history_resnet50_model_50epoch.h5", history_resnet50_model.history)

#### Evaluate the model

In [18]:
resnet50_model_loaded = load_model('resnet50_model_50epoch.h5')

In [20]:
resnet50_model_loaded.evaluate(train_generator_resnet50)

231/231 [==============================] - 383s 2s/step - loss: 0.8028 - accuracy: 0.6931 - auc: 0.9632 - precision: 0.7742 - recall: 0.5939


[0.8027752637863159,
 0.6930726766586304,
 0.9631621241569519,
 0.7742305397987366,
 0.5938979983329773]

In [21]:
resnet50_model_loaded.evaluate(test_generator_resnet50)

77/77 [==============================] - 127s 2s/step - loss: 0.6349 - accuracy: 0.7542 - auc: 0.9762 - precision: 0.7986 - recall: 0.6941


[0.6349151134490967,
 0.7541615962982178,
 0.9761607050895691,
 0.7986451983451843,
 0.6940722465515137]

In [22]:
history_resnet50_model_loaded = hdf5storage.loadmat("history_resnet50_model_50epoch.h5")

In [23]:
print(history_resnet50_model_loaded.keys())

dict_keys(['accuracy', 'auc', 'loss', 'precision', 'recall', 'val_accuracy', 'val_auc', 'val_loss', 'val_precision', 'val_recall'])


### Model tuning

In [27]:
resnet50_model_tune1 = tf.keras.applications.resnet50.ResNet50(include_top=False, 
                                                            weights = 'imagenet')
for layers in resnet50_model_tune1.layers[-1:]:
  layers.trainable = True

In [28]:
resnet50_model_tune1.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [29]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [30]:
resnet50_model_tune1 = custom_classifier(resnet50_model_tune1, num_classes)

In [31]:
resnet50_model_tune1.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [32]:
history_resnet50_model_tune1 = resnet50_model_tune1.fit(train_generator_resnet50,
                                                  steps_per_epoch=len(train_generator_resnet50),
                                                  epochs=50,
                                                  validation_data=val_generator_resnet50,
                                                  validation_steps=len(val_generator_resnet50))

Epoch 1/50
231/231 [==============================] - 1839s 8s/step - loss: 0.5923 - accuracy: 0.7966 - auc: 0.9784 - precision: 0.8500 - recall: 0.7496 - val_loss: 2.7617 - val_accuracy: 0.1248 - val_auc: 0.5546 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
231/231 [==============================] - 1818s 8s/step - loss: 0.2820 - accuracy: 0.9034 - auc: 0.9936 - precision: 0.9161 - recall: 0.8914 - val_loss: 1.8331 - val_accuracy: 0.3771 - val_auc: 0.8093 - val_precision: 0.5564 - val_recall: 0.0569
Epoch 3/50
231/231 [==============================] - 1791s 8s/step - loss: 0.2199 - accuracy: 0.9233 - auc: 0.9957 - precision: 0.9320 - recall: 0.9158 - val_loss: 0.9590 - val_accuracy: 0.6419 - val_auc: 0.9481 - val_precision: 0.7486 - val_recall: 0.5425
Epoch 4/50
231/231 [==============================] - 1797s 8s/step - loss: 0.2032 - accuracy: 0.9280 - auc: 0.9961 - precision: 0.9362 - recall: 0.9216 - val_loss: 0.3262 - val_accuracy: 0.8883 - val_auc: 0.9926 - val

In [33]:
resnet50_model.save('resnet50_model_tune1_50epoch.h5')

In [35]:
hdf5storage.savemat("history_resnet50_model_tune1_50epoch.h5", history_resnet50_model_tune1.history)